In [1]:
#https://www.kaggle.com/c/invasive-species-monitoring/data

In [2]:
from datetime import datetime
HOMEPATH = "/home/ubuntu/fastai/"
DATA_HOME_DIR = HOMEPATH + "data/invasive/"

import os

In [3]:
os.chdir(HOMEPATH)
print ("os.getcwd:", os.getcwd())
# Rather than importing everything manually, we'll make things easy
#   and load them all in utils.py, and just import them from there.
import utils;
from utils import *
from shutil import copyfile
import time

import numpy as np
import pandas as pd
from random import shuffle
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
%matplotlib inline

('os.getcwd:', '/home/ubuntu/fastai')


WARNING (theano.sandbox.cuda): The cuda backend is deprecated and will be removed in the next release (v0.10).  Please switch to the gpuarray backend. You can get more information about how to switch at this URL:
 https://github.com/Theano/Theano/wiki/Converting-to-the-new-gpu-back-end%28gpuarray%29

Using gpu device 0: Tesla K80 (CNMeM is disabled, cuDNN 5110)
Using Theano backend.


In [4]:
TRAIN  = DATA_HOME_DIR + "train/"
VALID  = DATA_HOME_DIR + "valid/"
TEST   = DATA_HOME_DIR + "test/"
RESULTS = DATA_HOME_DIR + "results/"
SAMPLE =  DATA_HOME_DIR + "sample/"
SAMPLE_TRAIN = SAMPLE + "train/"
SAMPLE_VALID = SAMPLE + "valid/"
SAMPLE_TEST = SAMPLE + "test/"
SAMPLE_RESULTS = SAMPLE + "results/"

In [5]:
#change this as appropriate between smaller sample data set(SAMPLE) and production data(DATA_HOME_DIR)
WORKING_DATA = SAMPLE
#WORKING_DATA = DATA_HOME_DIR 
#------------------future me : swap between above options. do not change below--------------
WORKING_TEST = WORKING_DATA + "test/" #We use all the test data
WORKING_TRAIN = WORKING_DATA + "train/"
WORKING_VALID = WORKING_DATA + "valid/"
WORKING_RESULTS = WORKING_DATA + "results/"

print ("WORKING_DATA:", WORKING_DATA)
print ("WORKING_TEST:", WORKING_TEST)
print ("WORKING_TRAIN:", WORKING_TRAIN)
print ("WORKING_VALID:", WORKING_VALID)
print ("WORKING_RESULTS:", WORKING_RESULTS)

('WORKING_DATA:', '/home/ubuntu/fastai/data/invasive/')
('WORKING_TEST:', '/home/ubuntu/fastai/data/invasive/test/')
('WORKING_TRAIN:', '/home/ubuntu/fastai/data/invasive/train/')
('WORKING_VALID:', '/home/ubuntu/fastai/data/invasive/valid/')
('WORKING_RESULTS:', '/home/ubuntu/fastai/data/invasive/results/')


In [6]:
# traverse root directory, and list directories as dirs and files as files
print (DATA_HOME_DIR)
for root, dirs, files in os.walk(DATA_HOME_DIR):
    path = root.split(os.sep)
    print (len(path) - 1) * '-', os.path.basename(root), len(files) 

/home/ubuntu/fastai/data/invasive/
------  6
------ valid 0
------- invasive 434
------- harmless 254
------ results 0
------ train 0
------- invasive 1014
------- harmless 593
------ test 0
------- unknown 1531
------ sample 0
------- valid 0
-------- invasive 86
-------- harmless 50
------- results 1
------- train 0
-------- invasive 202
-------- harmless 118
------- test 0
-------- unknown 306


In [7]:
#initiate standard Vgg16 model
from vgg16 import Vgg16
vgg = Vgg16()

In [8]:
#finetune model with our data
batches = vgg.get_batches(WORKING_TRAIN)
vgg.finetune(batches)

Found 1607 images belonging to 2 classes.


In [9]:
#set attributes
num_epochs = 4
lr = 0.001
import time

In [10]:
startTime= datetime.now()
print ("startTime:", startTime)

train_batches = vgg.get_batches(WORKING_TRAIN)
valid_batches = vgg.get_batches(WORKING_VALID)

timeElapsed=datetime.now()-startTime
print('Time elapsed (hh:mm:ss.ms) {}'.format(timeElapsed))


('startTime:', datetime.datetime(2017, 12, 12, 1, 7, 45, 315671))
Found 1607 images belonging to 2 classes.
Found 688 images belonging to 2 classes.
Time elapsed (hh:mm:ss.ms) 0:00:00.047381


In [11]:
type(train_batches)

keras.preprocessing.image.DirectoryIterator

In [ ]:
#from vgg16.py: def fit(self, batches, val_batches, nb_epoch=1)
vgg.fit(train_batches, valid_batches, nb_epoch=4)

Epoch 1/4
1607/1607 [==============================] - 90s - loss: 0.4780 - acc: 0.8270 - val_loss: 0.3486 - val_acc: 0.8866
Epoch 2/4
1296/1607 [=======================>......] - ETA: 10s - loss: 0.4111 - acc: 0.8681

In [ ]:
valid_batches, valid_preds = vgg.test(WORKING_VALID)
print ("valid_batches:", type(valid_batches))
valid_filenames = valid_batches.filenames
expected_labels = valid_batches.classes
print ("valid_filenames:", type(valid_filenames), len(valid_filenames))
print ("expected_labels:", type(expected_labels), len(expected_labels))
print "valid_filenames", valid_filenames[0:5]
print "expected_labels", expected_labels[0:5]
print "valid_batches.class_indices:", valid_batches.class_indices

In [ ]:
#quick visual inspection
valid_preds[:5]
#col 0 = predictions of harmless, col 1 = predictions of invasive
#nb: row sum = 1.0

In [ ]:
our_predictions = valid_preds[:,1]
print ("our_predictions:", our_predictions[0:5])
valid_pred_labels = np.round(valid_preds[:,1])
print ("valid_pred_labels:", valid_pred_labels[0:5])

In [ ]:
print valid_pred_labels.shape
valid_pred_labels[0:5]

In [ ]:
from sklearn.metrics import confusion_matrix
cm = confusion_matrix(expected_labels, valid_pred_labels)
plot_confusion_matrix(cm, valid_batches.class_indices)

In [ ]:
from sklearn.metrics import accuracy_score
accuracy_score(expected_labels, valid_pred_labels)

In [ ]:
#Number of images to view for each visualization task
n_view = 4

In [ ]:
from keras.preprocessing import image

#Helper function to plot images by index in the validation set 
#Plots is a helper function in utils.py
def plots_idx(idx, titles):
    plots([image.load_img(WORKING_VALID + valid_filenames[i]) for i in idx], titles=titles)

In [ ]:
def getFilenames(idx):
    filenames_ = []
    for i in idx:
        filenames_.append(valid_filenames[i])
    return filenames_

In [ ]:
def getProbs(idx):
    probs_ = []
    for i in idx:
        probs_.append(our_predictions[i])
    return probs_

In [ ]:
def plotSelected(correct, description):
    print("found %d %s labels" % (len(correct), description))
    idx = np.random.permutation(correct)
    titles = np.where(our_labels[idx[:n_view]] == 0.0, 'harmless', 'invasive')
    plots_idx(idx[:n_view], titles)
    print getFilenames(idx[:n_view])
    print getProbs(idx[:n_view])


In [ ]:
#selected at random: correctly predicted labels
correct = np.where(our_labels == expected_labels)[0]
plotSelected(correct, "correct")


In [ ]:
#selected at random: incorrectly predicted labels
correct = np.where(our_labels != expected_labels)[0]
plotSelected(correct, "incorrect")

In [ ]:
most_uncertain = np.argsort(abs(our_predictions-0.5))
plotSelected(most_uncertain[:n_view], "most uncertain")

In [ ]:
test_batches, test_preds = vgg.test(WORKING_TEST)

In [ ]:
test_filenames = test_batches.filenames

In [ ]:
ids = np.array([int(f[8:f.find('.')]) for f in test_filenames])
ids[0:10]

In [ ]:
is_invasive = test_preds[:,1]
print (is_invasive[0:5])

In [2]:
ts = str(int(time.time()))[-5:]
print "ts:", ts

ts: 41435


In [ ]:
submission = np.stack((ids, is_invasive), axis = 1)
filename = WORKING_RESULTS+"aspiringguru.csv"
np.savetxt(filename, submission, fmt="%d,%.3f", header='name,invasive', comments='')

In [ ]:
os.getcwd()